In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from gumbel_gen import train, is_next_five
from gumbel_softmax import GumbelSoftmax
from minigpt_utils import get_minigpt

import torch as t
import transformers

In [5]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
tokenizer._add_tokens(["[BEGIN]", "[END]"])
tokenizer.pad_token = "[END]"
tokenizer.eos_token = "[END]"

In [6]:
device = "cuda:5"
reward_fn = is_next_five      
gen_prompt = t.nn.functional.one_hot(t.tensor([[50257]], dtype=t.long, device=device), num_classes=50259)
eval_model = get_minigpt("model.pt").to(device)
reward_fn_args = {
    "eval_model": eval_model,
    "gen_prompt": gen_prompt,
    "head": 1,
    "layer": 2
}
gen_model = GumbelSoftmax(24, 50259).to(device)

In [49]:
gen_prompt

tensor([[[0, 0, 0,  ..., 0, 1, 0]]], device='cuda:5')

In [50]:
old_weight = gen_model.weight
old_weight

Parameter containing:
tensor([[0.6046, 0.4902, 0.3170,  ..., 0.3805, 0.9637, 0.3499],
        [0.5428, 0.6429, 0.5243,  ..., 0.3819, 0.5002, 0.6117],
        [0.5705, 0.2866, 0.1501,  ..., 0.4780, 0.6510, 0.1300],
        ...,
        [0.9498, 0.9781, 0.8452,  ..., 0.5473, 0.5813, 0.3555],
        [0.9396, 0.8012, 0.1983,  ..., 0.6398, 0.1899, 0.1121],
        [0.3003, 0.6404, 0.7624,  ..., 0.4014, 0.7323, 0.3423]],
       device='cuda:5', requires_grad=True)

In [51]:
train(gen_model, reward_fn, reward_fn_args, num_batches=11111, batch_size=16, logging=True, comet_tag=f"test-gumbel")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/gadzin1203/interp-viz/dc12721f2efb4cd3b06b30c63181f47f

100%|██████████| 11111/11111 [06:20<00:00, 29.19it/s]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/gadzin1203/interp-viz/dc12721f2efb4cd3b06b30c63181f47f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [1112]      : (0.09707468748092651, 10.621932029724121)
COMET INFO:     lr               : 0.001
COMET INFO:     reward [11111]   : (2.4270761059597135e-05, 0.9482730627059937)
COMET INFO:     temp [11111]     : (0.01017910179101791, 2.0)
COMET INFO:     val_loss [11111] : (0.058101125061511

In [227]:
gumbel_noise = -t.log(-t.log(t.rand((1,) + gen_model.weight.shape, device=device)))

In [52]:
tokenizer.batch_decode(t.argmax(gen_model(1, 0.001), dim=-1))

['&&& profitability worshiported issuance UT{ refiningWanthref correlations& <-SEC], investment branchingyoutu ({ quantify ISBN Mk']

In [53]:
is_next_five(gen_model(4, 1e-2), 1, 1, eval_model, gen_prompt)

(tensor([0.5073, 0.8991, 0.9663, 0.8237], device='cuda:5',
        grad_fn=<SelectBackward0>),
 {})

In [15]:
logits = eval_model(tokenizer.encode('[BEGIN] &&& profitability worshiported issuance UT{ refiningWanthref correlations& <-SEC], investment branchingyoutu ({ quantify ISBN Mk', return_tensors='pt').to(device))

In [17]:
t.softmax(logits[:,-1], dim=-1)[:,5]

tensor([0.5282], device='cuda:5', grad_fn=<SelectBackward0>)